## RL : reinforcement learning (강화학습)

심층 강화 학습 (deep reinforcement learning)에서 가장 중요한 두가지를 Markov decision process(MDP)와 함께 소개
- policy gradient(정책 그레디언트)
- DQN(deep Q-networks)  

### 보상을 최적화 하기 위한 학습

RL에서 agent는 observation(관측)을 하고 주어진 environment(환경)에서 action(행동)을 한다. 그리고 그 결과로 reward(보상)을 받는다.  
  agent의 목적은 reward의 장기적 기대치를 최대로 만드는 행동을 학습하는 것이다.

### Policy(정책)
- sw agent가 행동을 결정하기 위해 사용하는 알고리즘을 Policy라고 한다.
- 정책에는 무작위성이 포함되어 있어 stochastic policy라고 한다.

### CartPole 환경 생성

[실제 영상](https://www.youtube.com/watch?v=5Q14EjnOJZc)

In [47]:
import gym
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [14]:
env = gym.make("CartPole-v1")
# 환경 초기화
obs = env.reset()
print(f"cart의 수평위치 : {obs[0]}\ncart의 속도 : {obs[1]}\npole의 각도 : {obs[2]}\npole의 각 속도 : {obs[3]}")

cart의 수평위치 : -0.033523060381412506
cart의 속도 : -0.00010122294770553708
pole의 각도 : -0.0022249510511755943
pole의 각 속도 : 0.026952475309371948


In [4]:
for i in gym.envs.registry.all():
    print(i)

EnvSpec(CartPole-v0)
EnvSpec(CartPole-v1)
EnvSpec(MountainCar-v0)
EnvSpec(MountainCarContinuous-v0)
EnvSpec(Pendulum-v1)
EnvSpec(Acrobot-v1)
EnvSpec(LunarLander-v2)
EnvSpec(LunarLanderContinuous-v2)
EnvSpec(BipedalWalker-v3)
EnvSpec(BipedalWalkerHardcore-v3)
EnvSpec(CarRacing-v0)
EnvSpec(Blackjack-v1)
EnvSpec(FrozenLake-v1)
EnvSpec(FrozenLake8x8-v1)
EnvSpec(CliffWalking-v0)
EnvSpec(Taxi-v3)
EnvSpec(Reacher-v2)
EnvSpec(Pusher-v2)
EnvSpec(Thrower-v2)
EnvSpec(Striker-v2)
EnvSpec(InvertedPendulum-v2)
EnvSpec(InvertedDoublePendulum-v2)
EnvSpec(HalfCheetah-v2)
EnvSpec(HalfCheetah-v3)
EnvSpec(Hopper-v2)
EnvSpec(Hopper-v3)
EnvSpec(Swimmer-v2)
EnvSpec(Swimmer-v3)
EnvSpec(Walker2d-v2)
EnvSpec(Walker2d-v3)
EnvSpec(Ant-v2)
EnvSpec(Ant-v3)
EnvSpec(Humanoid-v2)
EnvSpec(Humanoid-v3)
EnvSpec(HumanoidStandup-v2)
EnvSpec(FetchSlide-v1)
EnvSpec(FetchPickAndPlace-v1)
EnvSpec(FetchReach-v1)
EnvSpec(FetchPush-v1)
EnvSpec(HandReach-v0)
EnvSpec(HandManipulateBlockRotateZ-v0)
EnvSpec(HandManipulateBlockRotateZ

In [15]:
env.render()

True

In [18]:
img = env.render(mode = "rgb_array")
img.shape # 높이, 너비, 채널(=3 r,g,b)

(800, 1200, 3)

In [19]:
# 가능한 행동은 0, 1 두가지
env.action_space

Discrete(2)

In [25]:
action =1
obs, reward, done, info = env.step(action)
print(f"obs : {obs}\nreward : {reward}\ndone : {done}\ninfo : {info}")

obs : [-0.02182006  0.5854182  -0.01820745 -0.8545304 ]
reward : 1.0
done : False
info : {}


**obs** 

새로운 관측값 \[cart 수평위치, cart 속도, pole 각도, pole 속도\] 양수면 오른쪽 음수면 왼쪽을 의미한다


**reward**  

현재 환경에선 어떤 행동을 해도 step 마다 1.0의 reward 획독, 목적은 최대한 오래동안 프로그램이 실행하는것


**done**  
True이면 episode가 끝난다. 막대가 너무 기울지거나 화면 밖으로 나가거나 200 step이 지나면 episode가 종료된다. 각 episode 마다 꼭 reset을 해줘야함

`done = bool(`  
`            x < -self.x_threshold`  
`            or x > self.x_threshold`  
`            or theta < -self.theta_threshold_radians`  
`            or theta > self.theta_threshold_radians`  
`        )`  

1. 각도가 ±12° 이상
2. 카트 위치가 ±2.4 이상(카트 중앙이 디스플레이 가장자리에 도달)
3. episode length가 500보다 큼(v0의 경우 200).


**info**

디버깅이나 추가 정보를 알려줌

### 간단한 hard coding
- pole이 왼쪽으로 기울면 cart를 왼쪽으로 가속
- pole이 오른쪽으로 기울면 cart를 오른쪾으로 가속

In [27]:
def basic_policy(obs):
    angle = obs[2]
    
    '''
    angle이 음수면 pole이 왼쪽으로 기울었음을 의미 > cart 왼쪽으로
    angle이 양수면 pole이 오른쪽으로 기울었음을 의미 > cart 오른쪽으로
    
    '''
    return 0 if angle < 0 else 1

totals = []
for episode in range(10):
    episode_rewards = 0
    obs = env.reset()
    for step in range(50):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [32]:
print("mean :",np.mean(totals))
print("std :",np.std(totals))
print("min :",np.min(totals))
print("max :",np.max(totals))

mean : 43.6
std : 4.1521078984053394
min : 38.0
max : 50.0


In [35]:
env.seed(42)

frames = []

obs = env.reset()
for step in range(200):
    img = env.render(mode="rgb_array")
    frames.append(img)
    action = basic_policy(obs)

    obs, reward, done, info = env.step(action)
    if done:
        break

In [44]:

def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

In [45]:
plot_animation(frames)

### Neural Network Policy
- CartPole 환경에서 가능한 행동은 오른쪽, 왼쪽 두가지이다. 
- Input에 대해 0(왼쪽)의 확률을 출력하고 확률이 p라고 가정하면 1(오른쪽)의 확률은 1-p가 된다.


In [51]:
n_input = 4 # cart 위치,속도 pole 각도,속도

model = nn.Sequential(
    nn.ELU(5),
    nn.Sigmoid(1)
)